<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pairwise-association-mining" data-toc-modified-id="Pairwise-association-mining-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pairwise association mining</a></span><ul class="toc-item"><li><span><a href="#Sets-or-lists-of-annotation-terms-per-protein" data-toc-modified-id="Sets-or-lists-of-annotation-terms-per-protein-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Sets or lists of annotation terms per protein</a></span></li><li><span><a href="#Generating-all-possible-pairwise-combinations-(that-are-present)" data-toc-modified-id="Generating-all-possible-pairwise-combinations-(that-are-present)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generating all possible pairwise combinations (that are present)</a></span></li><li><span><a href="#Create-all-possible-pairs-=>-high-memory-usage-and-unnecessary-pairs!" data-toc-modified-id="Create-all-possible-pairs-=>-high-memory-usage-and-unnecessary-pairs!-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Create all possible pairs =&gt; high memory usage and unnecessary pairs!</a></span></li><li><span><a href="#Merging-annotations" data-toc-modified-id="Merging-annotations-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Merging annotations</a></span></li><li><span><a href="#Propagate-pairs" data-toc-modified-id="Propagate-pairs-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Propagate pairs</a></span><ul class="toc-item"><li><span><a href="#Store-propagated-pairs-as-sets" data-toc-modified-id="Store-propagated-pairs-as-sets-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Store propagated pairs as sets</a></span></li><li><span><a href="#Store-propagated-pairs-as-lists" data-toc-modified-id="Store-propagated-pairs-as-lists-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Store propagated pairs as lists</a></span></li><li><span><a href="#Store-propagated-pairs-as-sorted-lists" data-toc-modified-id="Store-propagated-pairs-as-sorted-lists-1.5.3"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>Store propagated pairs as sorted lists</a></span></li><li><span><a href="#Propagated-pairs-dictionary-memory-requirements" data-toc-modified-id="Propagated-pairs-dictionary-memory-requirements-1.5.4"><span class="toc-item-num">1.5.4&nbsp;&nbsp;</span>Propagated pairs dictionary memory requirements</a></span></li></ul></li><li><span><a href="#Pairwise-assoc" data-toc-modified-id="Pairwise-assoc-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Pairwise assoc</a></span><ul class="toc-item"><li><span><a href="#1-pre-compute-propagated-pairs" data-toc-modified-id="1-pre-compute-propagated-pairs-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>1 pre-compute propagated pairs</a></span><ul class="toc-item"><li><span><a href="#Pre-computer-+-pairwise_dict-with-sets-+-no-convert-to-set-during-loop" data-toc-modified-id="Pre-computer-+-pairwise_dict-with-sets-+-no-convert-to-set-during-loop-1.6.1.1"><span class="toc-item-num">1.6.1.1&nbsp;&nbsp;</span>Pre-computer + pairwise_dict with sets + no convert to set during loop</a></span></li><li><span><a href="#Pre-computer-+-pairwise_dict-with-lists-+-convert-to-set-during-loop" data-toc-modified-id="Pre-computer-+-pairwise_dict-with-lists-+-convert-to-set-during-loop-1.6.1.2"><span class="toc-item-num">1.6.1.2&nbsp;&nbsp;</span>Pre-computer + pairwise_dict with lists + convert to set during loop</a></span></li></ul></li><li><span><a href="#3-on-the-fly-calculation-of-propagated-pairs" data-toc-modified-id="3-on-the-fly-calculation-of-propagated-pairs-1.6.2"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>3 on the fly calculation of propagated pairs</a></span></li><li><span><a href="#Generator" data-toc-modified-id="Generator-1.6.3"><span class="toc-item-num">1.6.3&nbsp;&nbsp;</span>Generator</a></span></li></ul></li><li><span><a href="#Effect-of-conditional-inside-versus-outside-loops" data-toc-modified-id="Effect-of-conditional-inside-versus-outside-loops-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Effect of conditional inside versus outside loops</a></span></li></ul></li></ul></div>

# Pairwise association mining

In [1]:
import ast
import numpy as np
import pandas as pd
import itertools
import sys

from pathlib import Path

from goscripts import obo_tools
from phppipy.mining import pairwise

import os
os.chdir('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis')
os.getcwd()

'/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis'

In [2]:
# https://goshippo.com/blog/measure-real-size-any-python-object/

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [3]:
input_file = "data/interim//35237/ppi_data/ppi-annotations.tsv"
obo_file = "data/raw/go_data/go.obo"

ppi_file = Path(input_file)
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
print('PPIs were imported from {}\n'.format(ppi_file))

PPIs were imported from data/interim/35237/ppi_data/ppi-annotations.tsv



In [4]:
for i in ['GO_xref_A', 'GO_xref_B', 'interpro_xref_A', 'interpro_xref_B']:
    ppi_df.loc[~ppi_df[i].isnull(), i] = ppi_df.loc[~ppi_df[i].isnull(),
                                                    i].map(ast.literal_eval)
    
merged_annotations_A = pairwise.merge_annotations(
    ppi_df, ['GO_xref_A', 'interpro_xref_A'])
labeled_merged_annotations_A = pairwise.add_hp_label(merged_annotations_A, 'h')
ppi_df['annotations_A'] = labeled_merged_annotations_A

merged_annotations_B = pairwise.merge_annotations(
    ppi_df, ['GO_xref_B', 'interpro_xref_B'])
labeled_merged_annotations_B = pairwise.add_hp_label(merged_annotations_B, 'p')
ppi_df['annotations_B'] = labeled_merged_annotations_B

In [5]:
ppi_df.columns

Index(['aliases_A', 'aliases_B', 'alt_identifiers_A', 'alt_identifiers_B',
       'author', 'confidence_score', 'detection_method', 'display_id_A',
       'display_id_B', 'interaction_identifiers', 'interaction_type', 'origin',
       'pathogen', 'publication', 'source_database_ids', 'taxid_A', 'taxid_B',
       'xref_A', 'xref_B', 'taxid_name_A', 'taxid_name_B',
       'xref_partners_sorted', 'GO_xref_A', 'GO_xref_B', 'interpro_xref_A',
       'interpro_xref_B', 'annotations_A', 'annotations_B'],
      dtype='object')

In [6]:
# create GO dictionary for propagation
obo_path = Path(obo_file)
go_dict = obo_tools.importOBO(obo_path, ignore_part_of=False)
obo_tools.buildGOtree(
    go_dict, root_nodes=['GO:0008150', 'GO:0005575', 'GO:0003674'])

Retrieved 47179 GO terms from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/go_data/go.obo 

Adding secondary GO identifiers...



In [7]:
columns = ['annotations_A', 'annotations_B']
column_A = columns[0]
column_B = columns[1]

## Sets or lists of annotation terms per protein

Almost a 10x difference in size of sets compared to lists.

In [8]:
get_size(ppi_df['annotations_A'])

27436061

In [9]:
get_size(ppi_df['annotations_A'].map(list))

5521869

In [10]:
get_size(ppi_df['GO_xref_A'])

23373395

In [11]:
%timeit ppi_df['GO_xref_A'].map(list)
%timeit ppi_df['GO_xref_A'].apply(list)

20.2 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.6 ms ± 4.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
get_size(ppi_df['GO_xref_A'].map(list))

4918089

## Generating all possible pairwise combinations (that are present)

The cartesian product cannot contain duplicates of the form `(GO:0016197, GO:0055036) vs (GO:0055036, GO:0016197)`. 

Consequently, calling `set()` on the `itertools.product()` output is not required. A sort can optionally be preformed to equalise different rows.

Storing sets inside a pandas dataframe increases the memory requirements, so a list is prefered.

In [13]:
all(ppi_df[columns].apply(lambda x: sorted(list(set([i for i in itertools.product(x[column_A], x[column_B])]))),axis=1) == ppi_df[columns].apply(lambda x: sorted(list([i for i in itertools.product(x[column_A], x[column_B])])),axis=1))

True

In [14]:
get_size(ppi_df[columns].apply(lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),axis=1))

226787095

In [15]:
get_size(ppi_df[columns].apply(lambda x: [i for i in itertools.product(x[column_A], x[column_B])],axis=1))

37543503

Sorting has a considerable time impact.

In [16]:
%timeit ppi_df[columns].apply(lambda x: [i for i in itertools.product(x[column_A], x[column_B])],axis=1)

867 ms ± 91.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit ppi_df[columns].apply(lambda x: sorted([i for i in itertools.product(x[column_A], x[column_B])]),axis=1)

2.44 s ± 476 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The cartesian product operates equally fast on lists and sets of annotation terms (likely due to the limited size per row).

In [18]:
ppi_df['annotations_A_list'] = ppi_df['annotations_A'].map(list)
ppi_df['annotations_B_list'] = ppi_df['annotations_B'].map(list)
%timeit ppi_df[columns].apply(lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),axis=1)
%timeit ppi_df[['annotations_A_list', 'annotations_B_list']].apply(lambda x: set([i for i in itertools.product(x['annotations_A_list'], x['annotations_B_list'])]),axis=1)

1.23 s ± 88.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.17 s ± 62.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
ppi_df['annotations_A'][0]

{'h@GO:0000166',
 'h@GO:0003723',
 'h@GO:0005515',
 'h@GO:0005524',
 'h@GO:0005737',
 'h@GO:0005829',
 'h@GO:0005832',
 'h@GO:0005874',
 'h@GO:0006457',
 'h@GO:0006458',
 'h@GO:0032212',
 'h@GO:0050821',
 'h@GO:0051082',
 'h@GO:0061077',
 'h@GO:0070062',
 'h@GO:0071987',
 'h@GO:1904851',
 'h@GO:1904871',
 'h@GO:1904874',
 'h@IPR002194',
 'h@IPR002423',
 'h@IPR012722',
 'h@IPR017998',
 'h@IPR027409',
 'h@IPR027410',
 'h@IPR027413'}

In [20]:
ppi_df['annotations_A_list'][0]

['h@GO:0005524',
 'h@GO:0070062',
 'h@GO:0005737',
 'h@IPR027413',
 'h@IPR027410',
 'h@IPR017998',
 'h@GO:0005832',
 'h@GO:0006457',
 'h@GO:1904874',
 'h@GO:1904851',
 'h@GO:0003723',
 'h@IPR002423',
 'h@GO:0050821',
 'h@GO:0000166',
 'h@GO:0071987',
 'h@GO:0005874',
 'h@GO:0051082',
 'h@GO:0061077',
 'h@GO:0006458',
 'h@GO:0005829',
 'h@IPR002194',
 'h@IPR027409',
 'h@GO:0032212',
 'h@GO:1904871',
 'h@GO:0005515',
 'h@IPR012722']

In [21]:
list(itertools.product(ppi_df['annotations_A_list'][0], ppi_df['annotations_B_list'][0]))

[('h@GO:0005524', 'p@GO:0016021'),
 ('h@GO:0005524', 'p@GO:0016020'),
 ('h@GO:0005524', 'p@IPR036157'),
 ('h@GO:0005524', 'p@IPR029054'),
 ('h@GO:0070062', 'p@GO:0016021'),
 ('h@GO:0070062', 'p@GO:0016020'),
 ('h@GO:0070062', 'p@IPR036157'),
 ('h@GO:0070062', 'p@IPR029054'),
 ('h@GO:0005737', 'p@GO:0016021'),
 ('h@GO:0005737', 'p@GO:0016020'),
 ('h@GO:0005737', 'p@IPR036157'),
 ('h@GO:0005737', 'p@IPR029054'),
 ('h@IPR027413', 'p@GO:0016021'),
 ('h@IPR027413', 'p@GO:0016020'),
 ('h@IPR027413', 'p@IPR036157'),
 ('h@IPR027413', 'p@IPR029054'),
 ('h@IPR027410', 'p@GO:0016021'),
 ('h@IPR027410', 'p@GO:0016020'),
 ('h@IPR027410', 'p@IPR036157'),
 ('h@IPR027410', 'p@IPR029054'),
 ('h@IPR017998', 'p@GO:0016021'),
 ('h@IPR017998', 'p@GO:0016020'),
 ('h@IPR017998', 'p@IPR036157'),
 ('h@IPR017998', 'p@IPR029054'),
 ('h@GO:0005832', 'p@GO:0016021'),
 ('h@GO:0005832', 'p@GO:0016020'),
 ('h@GO:0005832', 'p@IPR036157'),
 ('h@GO:0005832', 'p@IPR029054'),
 ('h@GO:0006457', 'p@GO:0016021'),
 ('h@GO:000

In [22]:
columns

['annotations_A', 'annotations_B']

Not calling `set()` on the final result also gives a (very) small speedup.

In [23]:
%timeit ppi_df[columns].apply(lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),axis=1)
%timeit ppi_df[columns].apply(lambda x: [i for i in itertools.product(x[column_A], x[column_B])],axis=1)

1.18 s ± 126 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
876 ms ± 67.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
ppi_df['annotation_pairs_set'] = ppi_df[columns].apply(lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),axis=1)
ppi_df['annotation_pairs'] = ppi_df[columns].apply(lambda x: [i for i in itertools.product(x[column_A], x[column_B])],axis=1)

In [25]:
ppi_df['annotation_pairs']

0        [(h@GO:0005524, p@GO:0016021), (h@GO:0005524, ...
1        [(h@GO:0006924, p@GO:0006351), (h@GO:0006924, ...
2        [(h@GO:0006272, p@GO:0034214), (h@GO:0006272, ...
3        [(h@IPR000504, p@GO:0006355), (h@IPR000504, p@...
4        [(h@GO:0070062, p@GO:0006414), (h@GO:0070062, ...
5        [(h@GO:0070062, p@GO:0006355), (h@GO:0070062, ...
6        [(h@GO:0003677, p@GO:0032508), (h@GO:0003677, ...
7        [(h@GO:0003677, p@GO:0032508), (h@GO:0003677, ...
8        [(h@GO:0003677, p@GO:0034214), (h@GO:0003677, ...
9        [(h@GO:0005737, p@GO:0032508), (h@GO:0005737, ...
10       [(h@GO:0003677, p@GO:0034214), (h@GO:0003677, ...
11       [(h@GO:0003684, p@GO:0003688), (h@GO:0003684, ...
12       [(h@GO:0006272, p@GO:0003688), (h@GO:0006272, ...
13       [(h@GO:0030866, p@GO:0039548), (h@GO:0030866, ...
14       [(h@IPR000387, p@GO:0039548), (h@IPR000387, p@...
15       [(h@GO:0000976, p@GO:0003688), (h@GO:0000976, ...
16       [(h@GO:0005737, p@IPR038575), (h@GO:0005737, p.

Add all present annotation term tuples to one list.

This code must join all tuples across all rows and remove duplicates.

In [26]:
%timeit set(itertools.chain.from_iterable(ppi_df['annotation_pairs']))

796 ms ± 61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit set(itertools.chain.from_iterable(ppi_df['annotation_pairs'].values))

801 ms ± 66.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit set().union(*ppi_df['annotation_pairs'])

777 ms ± 68.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The `union(*column)` operation will always result in the same set, regardless of initial order (as long as the tuples themselves are sorted h@-p@).

In [29]:
a = ppi_df.loc[0:10, 'annotation_pairs'].copy()
a.loc[1] = a.loc[0]
print(a.loc[0])

[('h@GO:0005524', 'p@GO:0016021'), ('h@GO:0005524', 'p@GO:0016020'), ('h@GO:0005524', 'p@IPR036157'), ('h@GO:0005524', 'p@IPR029054'), ('h@GO:0070062', 'p@GO:0016021'), ('h@GO:0070062', 'p@GO:0016020'), ('h@GO:0070062', 'p@IPR036157'), ('h@GO:0070062', 'p@IPR029054'), ('h@GO:0005737', 'p@GO:0016021'), ('h@GO:0005737', 'p@GO:0016020'), ('h@GO:0005737', 'p@IPR036157'), ('h@GO:0005737', 'p@IPR029054'), ('h@IPR027413', 'p@GO:0016021'), ('h@IPR027413', 'p@GO:0016020'), ('h@IPR027413', 'p@IPR036157'), ('h@IPR027413', 'p@IPR029054'), ('h@IPR027410', 'p@GO:0016021'), ('h@IPR027410', 'p@GO:0016020'), ('h@IPR027410', 'p@IPR036157'), ('h@IPR027410', 'p@IPR029054'), ('h@IPR017998', 'p@GO:0016021'), ('h@IPR017998', 'p@GO:0016020'), ('h@IPR017998', 'p@IPR036157'), ('h@IPR017998', 'p@IPR029054'), ('h@GO:0005832', 'p@GO:0016021'), ('h@GO:0005832', 'p@GO:0016020'), ('h@GO:0005832', 'p@IPR036157'), ('h@GO:0005832', 'p@IPR029054'), ('h@GO:0006457', 'p@GO:0016021'), ('h@GO:0006457', 'p@GO:0016020'), ('h@G

In [30]:
b = set().union(*a)

In [31]:
len(b)

3242

In [32]:
a2 = ppi_df.loc[0:10, 'annotation_pairs'].copy()
a2.loc[1] = a2.loc[0]
a2.loc[0] = a2.loc[0][::-1]
print(a2.loc[0])

[('h@IPR012722', 'p@IPR029054'), ('h@IPR012722', 'p@IPR036157'), ('h@IPR012722', 'p@GO:0016020'), ('h@IPR012722', 'p@GO:0016021'), ('h@GO:0005515', 'p@IPR029054'), ('h@GO:0005515', 'p@IPR036157'), ('h@GO:0005515', 'p@GO:0016020'), ('h@GO:0005515', 'p@GO:0016021'), ('h@GO:1904871', 'p@IPR029054'), ('h@GO:1904871', 'p@IPR036157'), ('h@GO:1904871', 'p@GO:0016020'), ('h@GO:1904871', 'p@GO:0016021'), ('h@GO:0032212', 'p@IPR029054'), ('h@GO:0032212', 'p@IPR036157'), ('h@GO:0032212', 'p@GO:0016020'), ('h@GO:0032212', 'p@GO:0016021'), ('h@IPR027409', 'p@IPR029054'), ('h@IPR027409', 'p@IPR036157'), ('h@IPR027409', 'p@GO:0016020'), ('h@IPR027409', 'p@GO:0016021'), ('h@IPR002194', 'p@IPR029054'), ('h@IPR002194', 'p@IPR036157'), ('h@IPR002194', 'p@GO:0016020'), ('h@IPR002194', 'p@GO:0016021'), ('h@GO:0005829', 'p@IPR029054'), ('h@GO:0005829', 'p@IPR036157'), ('h@GO:0005829', 'p@GO:0016020'), ('h@GO:0005829', 'p@GO:0016021'), ('h@GO:0006458', 'p@IPR029054'), ('h@GO:0006458', 'p@IPR036157'), ('h@GO:

In [33]:
b2 = set().union(*a2)

In [34]:
len(b2)

3242

Comparing the `union` method on lists vs sets.

In [35]:
%timeit set().union(*ppi_df['annotation_pairs_set'])

801 ms ± 76.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
pairs = set().union(*ppi_df['annotation_pairs'])
pairs_set = set().union(*ppi_df['annotation_pairs_set'])

In [37]:
list(pairs)[0:10]

[('h@IPR001382', 'p@GO:0039526'),
 ('h@IPR000266', 'p@GO:0030683'),
 ('h@GO:0030425', 'p@IPR036915'),
 ('h@GO:0007267', 'p@IPR003470'),
 ('h@IPR014401', 'p@GO:0005515'),
 ('h@GO:0098815', 'p@GO:0030430'),
 ('h@IPR008123', 'p@GO:0005515'),
 ('h@IPR033124', 'p@GO:0030430'),
 ('h@IPR036457', 'p@GO:0003688'),
 ('h@GO:0086064', 'p@GO:0039686')]

In [38]:
list(pairs_set)[0:10]

[('h@IPR001382', 'p@GO:0039526'),
 ('h@IPR000266', 'p@GO:0030683'),
 ('h@GO:0030425', 'p@IPR036915'),
 ('h@GO:0007267', 'p@IPR003470'),
 ('h@IPR014401', 'p@GO:0005515'),
 ('h@GO:0098815', 'p@GO:0030430'),
 ('h@IPR008123', 'p@GO:0005515'),
 ('h@IPR033124', 'p@GO:0030430'),
 ('h@IPR036457', 'p@GO:0003688'),
 ('h@GO:0086064', 'p@GO:0039686')]

This `union` method seems to work fine on lists (instead of sets).

In [39]:
sorted(pairs) == sorted(list(pairs_set))

True

In [40]:
len(pairs)

783261

In [41]:
sys.getsizeof(pairs) 

33554656

In [42]:
get_size(pairs)

96488097

In [43]:
sys.getsizeof(list(pairs)) 

7049456

In [44]:
get_size(list(pairs))

69982897

In [45]:
np.array(list(pairs)).nbytes

75193056

In [46]:
sys.getsizeof(np.array(list(pairs)))

75193168

In [47]:
get_size(np.array(list(pairs)))

75193737

## Create all possible pairs => high memory usage and unnecessary pairs!

In [ ]:
%timit list(itertools.combinations(set().union(*ppi_df[columns].values.ravel('K')), 2))

In [ ]:
pairs_all = list(itertools.combinations(set().union(*ppi_df[columns].values.ravel('K')), 2))

In [17]:
len(pairs_all)

61654960

In [20]:
sys.getsizeof(pairs_all) 

509119928

In [21]:
del pairs_all

In [47]:
list(pairs)[0]

('GO:0000209', 'GO:0060326')

## Merging annotations

In [48]:
ppi_df[columns]

,annotations_A,annotations_B
0,"{h@GO:0005524, h@GO:0070062, h@GO:0005737, h@I...","{p@GO:0016021, p@GO:0016020, p@IPR036157, p@IP..."
1,"{h@GO:0006924, h@IPR022773, h@GO:0005737, h@GO...","{p@GO:0006351, p@GO:0039650, p@GO:0039503, p@G..."
2,"{h@GO:0006272, h@GO:0006303, h@IPR012337, h@GO...","{p@GO:0034214, p@GO:0003677, p@GO:0006260, p@G..."
3,"{h@IPR000504, h@GO:0005737, h@GO:0045727, h@GO...","{p@GO:0006355, p@IPR007031, p@GO:0006351, p@GO..."
4,"{h@GO:0070062, h@GO:0005737, h@GO:0016020, h@I...","{p@GO:0006414, p@GO:0030430, p@GO:0006355, p@G..."
5,"{h@GO:0070062, h@GO:0005737, h@GO:0016020, h@I...","{p@GO:0006355, p@IPR007031, p@GO:0006351, p@GO..."
6,"{h@GO:0003677, h@GO:0045444, h@GO:0065004, h@G...","{p@GO:0032508, p@IPR016393, p@GO:0016787, p@IP..."
7,"{h@GO:0003677, h@GO:0045893, h@GO:0061665, h@G...","{p@GO:0032508, p@IPR016393, p@GO:0016787, p@IP..."
8,"{h@GO:0003677, h@GO:0045444, h@GO:0065004, h@G...","{p@GO:0034214, p@GO:0003677, p@GO:0006260, p@G..."
9,"{h@GO:0005737, h@GO:0016020, h@GO:0005642, h@G...","{p@GO:0032508, p@IPR016393, p@GO:0016787, p@IP..."


In [49]:
%timeit ppi_df[columns].apply(lambda x: set().union(*x), axis=1)

266 ms ± 30 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
ppi_df[columns].apply(lambda x: set().union(*x), axis=1).loc[0]

{'h@GO:0000166',
 'h@GO:0003723',
 'h@GO:0005515',
 'h@GO:0005524',
 'h@GO:0005737',
 'h@GO:0005829',
 'h@GO:0005832',
 'h@GO:0005874',
 'h@GO:0006457',
 'h@GO:0006458',
 'h@GO:0032212',
 'h@GO:0050821',
 'h@GO:0051082',
 'h@GO:0061077',
 'h@GO:0070062',
 'h@GO:0071987',
 'h@GO:1904851',
 'h@GO:1904871',
 'h@GO:1904874',
 'h@IPR002194',
 'h@IPR002423',
 'h@IPR012722',
 'h@IPR017998',
 'h@IPR027409',
 'h@IPR027410',
 'h@IPR027413',
 'p@GO:0016020',
 'p@GO:0016021',
 'p@IPR029054',
 'p@IPR036157'}

In [58]:
merged_annotations = pairwise.merge_annotations(ppi_df, columns)

In [52]:
merged_annotations[0]

{'h@GO:0000166',
 'h@GO:0003723',
 'h@GO:0005515',
 'h@GO:0005524',
 'h@GO:0005737',
 'h@GO:0005829',
 'h@GO:0005832',
 'h@GO:0005874',
 'h@GO:0006457',
 'h@GO:0006458',
 'h@GO:0032212',
 'h@GO:0050821',
 'h@GO:0051082',
 'h@GO:0061077',
 'h@GO:0070062',
 'h@GO:0071987',
 'h@GO:1904851',
 'h@GO:1904871',
 'h@GO:1904874',
 'h@IPR002194',
 'h@IPR002423',
 'h@IPR012722',
 'h@IPR017998',
 'h@IPR027409',
 'h@IPR027410',
 'h@IPR027413',
 'p@GO:0016020',
 'p@GO:0016021',
 'p@IPR029054',
 'p@IPR036157'}

In [53]:
get_size(merged_annotations)

26553111

In [54]:
get_size(merged_annotations.map(list))

7363991

go_dict recursive_children already supplies a set, so no need to convert this back.

In [55]:
go_dict['GO:0005874'].recursive_children

{'GO:0000235',
 'GO:0005827',
 'GO:0005828',
 'GO:0005876',
 'GO:0005879',
 'GO:0005880',
 'GO:0005881',
 'GO:0010005',
 'GO:0020025',
 'GO:0033289',
 'GO:0035371',
 'GO:0036449',
 'GO:0055028',
 'GO:0061673',
 'GO:0097649',
 'GO:0097650',
 'GO:1901588',
 'GO:1903754',
 'GO:1904511',
 'GO:1905721',
 'GO:1905759',
 'GO:1905760',
 'GO:1990498',
 'GO:1990537',
 'GO:1990574',
 'GO:1990719',
 'GO:1990720',
 'GO:1990752',
 'GO:1990941'}

## Propagate pairs

Store propagated pairs as lists.

In [58]:
%timeit [len(pair) for pair in pairs]
%timeit [len(pair) for pair in list(pairs)]

123 ms ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
148 ms ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [57]:
%timeit for pair in list(pairs)[:1000]:    pairwise._propagate_pair(pair, go_dict)

125 ms ± 9.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Store propagated pairs as sets

In [49]:
def _propagate_pair_set(pair, go_dict):
    # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
    propagated_pair = []
    # for both annotations in the pair, propagate through GO hierarchy
    for term in pair:
        # only for GO terms, not IPR
        if 'GO' in term:
            prefix = term[:2]
            go_object = go_dict.get(term[2:])
            # append original annotation if it can't be found in GO dict
            if not go_object:
                propagated_pair.append({term})
            else:
                # store all child terms of parent term in a list
                child_terms = [
                    prefix + i for i in go_object.recursive_children
                ]
                # add parent term itself and remove duplicates
                propagation_set = set(child_terms) | set([term])
                # add propagated annotations to storage list
                propagated_pair.append(propagation_set)
        else:
            # store original term if it's not a GO term
            propagated_pair.append({term})

    # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
    # e.g. ('h@GO:0060384', 'p@GO:0016787') -> (['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'], ['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'])
    return tuple(propagated_pair)

In [62]:
%timeit for pair in list(pairs)[:1000]:    _propagate_pair_set(pair, go_dict)

77.6 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Store propagated pairs as lists

In [50]:
def _propagate_pair_list1(pair, go_dict):
    # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
    propagated_pair = []
    # for both annotations in the pair, propagate through GO hierarchy
    for term in pair:
        # only for GO terms, not IPR
        if 'GO' in term:
            prefix = term[:2]
            go_object = go_dict.get(term[2:])
            # append original annotation if it can't be found in GO dict
            if not go_object:
                propagated_pair.append([term])
            else:
                # store all child terms of parent term in a list
                child_terms = [
                    prefix + i for i in go_object.recursive_children
                ]
                # add parent term
                child_terms.append(term)
                # remove duplicates, convert to list and sort
                propagation_list = list(set(child_terms))
                # add propagated annotations to storage list
                propagated_pair.append(propagation_list)
        else:
            # store original term if it's not a GO term
            propagated_pair.append([term])

    # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
    # e.g. ('h@GO:0060384', 'p@GO:0016787') -> (['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'], ['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'])
    return tuple(propagated_pair)

In [64]:
%timeit for pair in list(pairs)[:1000]:    _propagate_pair_list1(pair, go_dict)

73.1 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Store propagated pairs as sorted lists

In [51]:
def _propagate_pair_list2(pair, go_dict):
    # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
    propagated_pair = []
    # for both annotations in the pair, propagate through GO hierarchy
    for term in pair:
        # only for GO terms, not IPR
        if 'GO' in term:
            prefix = term[:2]
            go_object = go_dict.get(term[2:])
            # append original annotation if it can't be found in GO dict
            if not go_object:
                propagated_pair.append([term])
            else:
                # store all child terms of parent term in a list
                child_terms = [
                    prefix + i for i in go_object.recursive_children
                ]
                # add parent term
                child_terms.append(term)
                # remove duplicates, convert to list and sort
                propagation_list = sorted(set(child_terms))
                # add propagated annotations to storage list
                propagated_pair.append(propagation_list)
        else:
            # store original term if it's not a GO term
            propagated_pair.append([term])

    # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
    # e.g. ('h@GO:0060384', 'p@GO:0016787') -> (['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'], ['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'])
    return tuple(propagated_pair)

In [66]:
%timeit for pair in list(pairs)[:1000]:    _propagate_pair_list2(pair, go_dict)

121 ms ± 7.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Propagated pairs dictionary memory requirements

The propagated dict can take up to 11-25 GB of RAM when it is structured as a dictionary of tuples pointing to sets.

In [52]:
def _propagate_pairs_set(pairs, go_dict):
    # create empty dictionary to store the two sets of propagated annotations
    propagated_pairs = {}
    # propagated_pairs = []

    # loop through sets of annotation pairs
    for pair in list(pairs):
        # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
        new_pair = []
        # for both annotations in the pair, propagate through GO hierarchy
        for term in pair:
            # only for GO terms, not IPR
            if 'GO' in term:
                prefix = term[:2]
                go_object = go_dict.get(term[2:])
                # append original annotation if it can't be found in GO dict
                if not go_object:
                    # store original term if no GO info available
                    new_pair.append({term})
                else:
                    # store all child terms of parent term in a list
                    child_terms = [
                        prefix + i for i in go_object.recursive_children
                    ]
                    # add parent term itself and remove duplicates
                    propagation_set = set(child_terms) | set([term])
                    # add propagated annotations to storage list
                    new_pair.append(propagation_set)
            else:
                # store original term if it's not a GO term
                new_pair.append({term})

        # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
        # e.g. ('h@GO:0060384', 'p@GO:0016787') -> ({'h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'}, {'h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'})
        propagated_pairs[pair] = tuple(new_pair)

    return propagated_pairs

In [53]:
pairs_dict = _propagate_pairs_set(list(pairs)[:100], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100 / 10**6)

3168777
24819.794417970003


In [87]:
pairs_dict = _propagate_pairs_set(list(pairs)[:100000], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100000 / 10**6)

2166206902
16967.05384267422


Previous values...due to random order inside pairs set?

In [120]:
pairs_dict = pairwise._propagate_pairs(list(pairs)[:100], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100 / 10**6)

1452605
11377.68844905


In [123]:
pairs_dict = _propagate_pairs_set(list(pairs)[:100000], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100000 / 10**6)

2192436790
17172.5023257219


In [112]:
get_size(pairs_dict[(('h@GO:0006911', 'p@GO:0006355'))])

70411

A set takes almost double the memory of a list.

In [60]:
get_size({'h@GO:0000166', 'h@GO:0003723', 'h@GO:0005515', 'h@GO:0005524', 'h@GO:0005737', 'h@GO:0005829', 'h@GO:0005832', 'h@GO:0005874', 'h@GO:0006457', 'h@GO:0006458', 'h@GO:0032212', 'h@GO:0050821', 'h@GO:0051082', 'h@GO:0061077', 'h@GO:0070062', 'h@GO:0071987', 'h@GO:1904851', 'h@GO:1904871', 'h@GO:1904874', 'h@IPR002194', 'h@IPR002423', 'h@IPR012722', 'h@IPR017998', 'h@IPR027409', 'h@IPR027410', 'h@IPR027413'
         })

3851

In [59]:
get_size(list({'h@GO:0000166', 'h@GO:0003723', 'h@GO:0005515', 'h@GO:0005524', 'h@GO:0005737', 'h@GO:0005829', 'h@GO:0005832', 'h@GO:0005874', 'h@GO:0006457', 'h@GO:0006458', 'h@GO:0032212', 'h@GO:0050821', 'h@GO:0051082', 'h@GO:0061077', 'h@GO:0070062', 'h@GO:0071987', 'h@GO:1904851', 'h@GO:1904871', 'h@GO:1904874', 'h@IPR002194', 'h@IPR002423', 'h@IPR012722', 'h@IPR017998', 'h@IPR027409', 'h@IPR027410', 'h@IPR027413'
         }))

1923

In [54]:
def _propagate_pairs_list(pairs, go_dict):
    # create empty dictionary to store the two sets of propagated annotations
    propagated_pairs = {}
    # propagated_pairs = []

    # loop through sets of annotation pairs
    for pair in list(pairs):
        # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
        new_pair = []
        # for both annotations in the pair, propagate through GO hierarchy
        for term in pair:
            # only for GO terms, not IPR
            if 'GO' in term:
                prefix = term[:2]
                go_object = go_dict.get(term[2:])
                # append original annotation if it can't be found in GO dict
                if not go_object:
                    # store original term if no GO info available
                    new_pair.append([term])
                else:
                    # store all child terms of parent term in a list
                    child_terms = [
                        prefix + i for i in go_object.recursive_children
                    ]
                    # add parent term itself and remove duplicates
                    propagation_set = list(set(child_terms) | set([term]))
                    # add propagated annotations to storage list
                    new_pair.append(propagation_set)
            else:
                # store original term if it's not a GO term
                new_pair.append([term])

        # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
        # e.g. ('h@GO:0060384', 'p@GO:0016787') -> ({'h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'}, {'h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'})
        propagated_pairs[pair] = tuple(new_pair)

    return propagated_pairs

In [55]:
pairs_dict = _propagate_pairs_list(list(pairs)[:100], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100 / 10**6)

1963849
15382.06331589


In [62]:
pairs_dict = _propagate_pairs_list(list(pairs)[:100000], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100000 / 10**6)

1364941334
10691.05314210174


In [63]:
del pairs_dict

**Conclusion: storing the propagated pairs as lists is a major reduction in size, however, doing this forces a `set()` operation to take place during the pairwise association mining procedure.**

## Pairwise assoc
Comparing the performance of pairwise association mining when

1. pre-computing the list of propagated pairs
2. computing the list as an iterator
3. computing the propagated pair on the fly inside the loop
4. Using set objects as input
5. Converting list objects to sets inside the loop

### 1 pre-compute propagated pairs

#### Pre-computer + pairwise_dict with sets + no convert to set during loop

In [56]:
def count_presences_propagated(pair_prop, interaction_dataframe, column_A,
                               column_B, merged_annotations):
    set_A = pair_prop[0]
    set_B = pair_prop[1]

    pair_count = np.sum(
        merged_annotations.map(
            lambda x: not x.isdisjoint(set_A) and not x.isdisjoint(set_B)
        ))
    label_one_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: not x.isdisjoint(set_A)
    ) & interaction_dataframe[column_B].map(
        lambda x: x.isdisjoint(set_B)))
    label_two_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: x.isdisjoint(set_A)
    ) & interaction_dataframe[column_B].map(
        lambda x: not x.isdisjoint(set_B)))
    label_one_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(set_A)))
    label_two_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(set_B)))
    absent_count = np.sum(
        merged_annotations.map(lambda x: x.isdisjoint(set_A)) &
        merged_annotations.map(lambda x: x.isdisjoint(set_B)))
    total_count = interaction_dataframe.shape[0]
    return pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count

In [59]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
pairs_dict = _propagate_pairs_set(list(pairs)[:100], go_dict)
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

7.17 s ± 421 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Pre-computer + pairwise_dict with lists + convert to set during loop

In [92]:
def count_presences_propagated(pair_prop, interaction_dataframe, column_A,
                               column_B, merged_annotations):
    set_A = set(pair_prop[0])
    set_B = set(pair_prop[1])

    pair_count = np.sum(
        merged_annotations.map(
            lambda x: not x.isdisjoint(set_A) and not x.isdisjoint(set_B)
        ))
    label_one_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: not x.isdisjoint(set_A)
    ) & interaction_dataframe[column_B].map(
        lambda x: x.isdisjoint(set_B)))
    label_two_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: x.isdisjoint(set_A)
    ) & interaction_dataframe[column_B].map(
        lambda x: not x.isdisjoint(set_B)))
    label_one_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(set_A)))
    label_two_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(set_B)))
    absent_count = np.sum(
        merged_annotations.map(lambda x: x.isdisjoint(set_A)) &
        merged_annotations.map(lambda x: x.isdisjoint(set_B)))
    total_count = interaction_dataframe.shape[0]
    return pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count

In [67]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
pairs_dict = _propagate_pairs_list(list(pairs)[:100], go_dict)
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
#     propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

7.35 s ± 260 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**There's next to no additional cost to converting lists to sets inside the mining function, so it is better to store lists inside the propagated dictionary to reduce memory requirements.**

In [65]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

1min 12s ± 3.95 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Splitting up the generation and mining loop.**

In [74]:
%%timeit
pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)

126 ms ± 8.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [75]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

7.06 s ± 269 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**More detailed profiling of time spent during function...**

In [62]:
%%prun
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

In [64]:
%load_ext line_profiler
%load_ext heat

In [68]:
def fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
    results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
    pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
    counter = 0
    for pair, propagated_pair in pairs_dict.items():
        counter += 1
        if counter % 10000 == 0:
            print('Processed {} lines out of {}'.format(
                counter, len(pairs)))
        # convert lists of annotations to sets in order to speed up comparisons
#         propagated_pair = list(set(i) for i in propagated_pair)
        pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
            propagated_pair, ppi_df, column_A, column_B,
            merged_annotations)
        pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
%lprun -f fast fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations)

```
Timer unit: 1e-06 s

Total time: 91.7701 s
File: <ipython-input-68-d38c677a9b5a>
Function: fast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
     2         1          2.0      2.0      0.0      results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
     3         1     154876.0 154876.0      0.2      pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
     4         1          2.0      2.0      0.0      counter = 0
     5      1001       2223.0      2.2      0.0      for pair, propagated_pair in pairs_dict.items():
     6      1000        832.0      0.8      0.0          counter += 1
     7      1000        939.0      0.9      0.0          if counter % 10000 == 0:
     8                                                       print('Processed {} lines out of {}'.format(
     9                                                           counter, len(pairs)))
    10                                                   # convert lists of annotations to sets in order to speed up comparisons
    11                                           #         propagated_pair = list(set(i) for i in propagated_pair)
    12      1000        885.0      0.9      0.0          pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
    13      1000        701.0      0.7      0.0              propagated_pair, ppi_df, column_A, column_B,
    14      1000   88386176.0  88386.2     96.3              merged_annotations)
    15      1000    3223449.0   3223.4      3.5          pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
    
```

---
Old version still converted the propagated pair sets to list, which is no longer required.

```
Timer unit: 1e-06 s

Total time: 95.401 s
File: <ipython-input-65-09a22af8a058>
Function: fast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
     2         1          2.0      2.0      0.0      results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
     3         1     169316.0 169316.0      0.2      pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
     4         1          2.0      2.0      0.0      counter = 0
     5      1001       8571.0      8.6      0.0      for pair, propagated_pair in pairs_dict.items():
     6      1000       1013.0      1.0      0.0          counter += 1
     7      1000       1125.0      1.1      0.0          if counter % 10000 == 0:
     8                                                       print('Processed {} lines out of {}'.format(
     9                                                           counter, len(pairs)))
    10                                                   # convert lists of annotations to sets in order to speed up comparisons
    11      1000      22910.0     22.9      0.0          propagated_pair = list(set(i) for i in propagated_pair)
    12      1000       1061.0      1.1      0.0          pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
    13      1000        730.0      0.7      0.0              propagated_pair, ppi_df, column_A, column_B,
    14      1000   91837815.0  91837.8     96.3              merged_annotations)
    15      1000    3358479.0   3358.5      3.5          pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
    ```

### 3 on the fly calculation of propagated pairs

On the fly propagation is an order of magnitude slower! 

7.06 s ± 269 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

versus

1 minute!

However, if the separate propagation step is counted, the difference vanishes:

1min 12s ± 3.95 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [71]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
counter = 0
for pair in list(pairs)[:1000]:
        propagated_pair = _propagate_pair_list1(pair, go_dict)
        counter += 1
        if counter % 10000 == 0:
            print('Processed {} lines out of {}'.format(
                counter, len(pairs)))
        propagated_pair = list(set(i) for i in propagated_pair)
        pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
            propagated_pair, ppi_df, column_A, column_B,
            merged_annotations)

        pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

1min 9s ± 2.45 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
def mem(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
    results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
    counter = 0
    for pair in list(pairs)[:1000]:
            propagated_pair = _propagate_pair_list1(pair, go_dict)
            counter += 1
            if counter % 10000 == 0:
                print('Processed {} lines out of {}'.format(
                    counter, len(pairs)))
            propagated_pair = list(set(i) for i in propagated_pair)
            pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
                propagated_pair, ppi_df, column_A, column_B,
                merged_annotations)

            pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
%lprun -f mem mem(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations)

```
Timer unit: 1e-06 s

Total time: 97.6505 s
File: <ipython-input-73-ed576d8dea15>
Function: mem at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def mem(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
     2         1          3.0      3.0      0.0      results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
     3         1          2.0      2.0      0.0      counter = 0
     4      1001      56425.0     56.4      0.1      for pair in list(pairs)[:1000]:
     5      1000     114686.0    114.7      0.1              propagated_pair = _propagate_pair_list1(pair, go_dict)
     6      1000       1170.0      1.2      0.0              counter += 1
     7      1000       1101.0      1.1      0.0              if counter % 10000 == 0:
     8                                                           print('Processed {} lines out of {}'.format(
     9                                                               counter, len(pairs)))
    10      1000      11713.0     11.7      0.0              propagated_pair = list(set(i) for i in propagated_pair)
    11      1000        883.0      0.9      0.0              pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
    12      1000        762.0      0.8      0.0                  propagated_pair, ppi_df, column_A, column_B,
    13      1000   94003924.0  94003.9     96.3                  merged_annotations)
    14                                           
    15      1000    3459817.0   3459.8      3.5              pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
```

### Generator

A generator is equall fast as the other two options, but it doesn't store the object in memory. Clear winner.

In [88]:
def prop_iterator(pairs, go_dict):
    # loop through sets of annotation pairs
    for pair in pairs:
        propagated_pair = _propagate_pair_list1(pair, go_dict)
        yield pair, tuple(propagated_pair)

In [97]:
for pair, propagated_pair in prop_iterator(list(pairs)[:2], go_dict):
    print(pair, propagated_pair)

('h@IPR001382', 'p@GO:0039526') (['h@IPR001382'], ['p@GO:0039651', 'p@GO:0039650', 'p@GO:0039526', 'p@GO:0019050', 'p@GO:0019051'])
('h@IPR000266', 'p@GO:0030683') (['h@IPR000266'], ['p@GO:0039504', 'p@GO:0039591', 'p@GO:0039503', 'p@GO:0039537', 'p@GO:0039723', 'p@GO:0039539', 'p@GO:0039722', 'p@GO:0039502', 'p@GO:0039672', 'p@GO:0039548', 'p@GO:0030683', 'p@GO:0039673', 'p@GO:0039580', 'p@GO:0039699', 'p@GO:0039574', 'p@GO:0046774', 'p@GO:0039557', 'p@GO:0039588', 'p@GO:0039724', 'p@GO:0039505', 'p@GO:0039579', 'p@GO:0039671', 'p@GO:0039545', 'p@GO:0039573', 'p@GO:0046776', 'p@GO:0039540', 'p@GO:0039538', 'p@GO:0039560', 'p@GO:0039554'])


In [96]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
counter = 0
for pair, propagated_pair in prop_iterator(list(pairs)[:1000], go_dict):
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

1min 5s ± 1.53 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [98]:
results_dict

{'pmi': {('h@IPR001382', 'p@GO:0039526'): 1.2298782593312223,
  ('h@IPR000266', 'p@GO:0030683'): 0.4171676644147274,
  ('h@GO:0030425', 'p@IPR036915'): 0.8424079086838426,
  ('h@GO:0007267', 'p@IPR003470'): 0.5390360128749159,
  ('h@IPR014401', 'p@GO:0005515'): 0.4738220338312717},
 'G': {('h@IPR001382', 'p@GO:0039526'): {'G': 0.6964506858863668,
   'p-value': 0.4039788914325656,
   'df': 1,
   'exp': array([[4.26353422e-01, 2.08657365e+03],
          [2.57364658e+00, 1.25954264e+04]])},
  ('h@IPR000266', 'p@GO:0030683'): {'G': 0.228019408326126,
   'p-value': 0.6329963119819196,
   'df': 1,
   'exp': array([[1.49778686e+00, 4.39750221e+03],
          [3.50221314e+00, 1.02824978e+04]])},
  ('h@GO:0030425', 'p@IPR036915'): {'G': 0.5996225730576674,
   'p-value': 0.4387220677981275,
   'df': 1,
   'exp': array([[1.11542390e+00, 2.48845761e+01],
          [6.28884576e+02, 1.40301154e+04]])},
  ('h@GO:0007267', 'p@IPR003470'): {'G': 0.3956444220808353,
   'p-value': 0.5293472753277543,
   

## Effect of conditional inside versus outside loops

Barely any effect at all.

In [99]:
prop = True
fast = True

In [101]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
counter = 0
for pair, propagated_pair in prop_iterator(list(pairs)[:1000], go_dict):
    if prop:
        if fast:
            counter += 1
            if counter % 10000 == 0:
                print('Processed {} lines out of {}'.format(
                    counter, len(pairs)))
            # convert lists of annotations to sets in order to speed up comparisons
            propagated_pair = list(set(i) for i in propagated_pair)
            pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
                propagated_pair, ppi_df, column_A, column_B,
                merged_annotations)
            pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
        else:
            counter += 1
            if counter % 10000 == 0:
                print('Processed {} lines out of {}'.format(
                    counter, len(pairs)))
            # convert lists of annotations to sets in order to speed up comparisons
            propagated_pair = list(set(i) for i in propagated_pair)
            pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
                propagated_pair, ppi_df, column_A, column_B,
                merged_annotations)
            pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
    else:
        counter += 1
        if counter % 10000 == 0:
            print('Processed {} lines out of {}'.format(
                counter, len(pairs)))
        # convert lists of annotations to sets in order to speed up comparisons
        propagated_pair = list(set(i) for i in propagated_pair)
        pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
            propagated_pair, ppi_df, column_A, column_B,
            merged_annotations)
        pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)


1min 4s ± 4.14 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
